In [1]:
import os

In [2]:
os.getcwd()

'H:\\document_classification\\Dataset'

In [3]:
os.chdir(r'H:\document_classification\Dataset\training-data')

In [4]:
os.getcwd()

'H:\\document_classification\\Dataset\\training-data'

In [5]:
import glob

In [6]:
filenames=glob.glob('*.txt')

In [9]:
#Creating the training dataframe
path="H:\\document_classification\\Dataset\\training-data"
training=[]
for i in range(len(filenames)):
    new_path=path+'\\'+filenames[i]
    with open(new_path,'r') as infile:
        data=infile.read()
        data=data.split('\n')
        if(filenames[i][0]=='a'):
            train=[words.split("\t")[1] for words in data if (words!='')&(words!='-----')]
        else:
            train=[words for words in data if (words!='')&(words!='-----')]
        training.append(train)

In [10]:
#Creating the training output 
import numpy as np
import re
y=[]
for i in range(0,len(training)):
    temp=[]
    for j in range(len(training[i])):
        training[i][j]=re.sub('[^A-Za-z0-9]',' ',training[i][j])
        s=[j+1]*len(training[i][j].split())
        temp.extend(s)
        k=np.array(temp)
        k=k.reshape(len(k),1)
        from sklearn.preprocessing import OneHotEncoder
        onehot_encoder = OneHotEncoder(sparse=False)
        k = onehot_encoder.fit_transform(k)
    #k=pad_sequences()
    y.append(k)
print(len(y))

2234


In [11]:
#(each record belongs to one of the 10 segments)
temp2=[]
for i in range(0,len(y)):
    temp=[]
    for j in range(len(y[i])):
        temp1=list(y[i][j])
        temp1.extend([0]*(10-len(temp1)))
        temp.append(temp1)
    temp2.append(temp)

In [12]:
y=temp2

In [13]:
#Loading the word embedding model
file = r"H:\\glove.6B.100d.txt"
import numpy as np
def loadGloveModel(gloveFile):
    print ("Loading Glove Model")
    with open(gloveFile, encoding="utf8" ) as f:
       content = f.readlines()
    model = {}
    for line in content:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print ("Done.",len(model)," words loaded!")
    return model
model_dict= loadGloveModel(file) 

Loading Glove Model
Done. 400000  words loaded!


In [ ]:
#Converting the word into vector
train=[]
for i in range(0,len(training)):
    temp1=[]
    for j in range(len(training[i])):
        training[i][j]=training[i][j].lower()
        #training[i][j]=re.sub('[^A-Za-z0-9]',' ',training[i][j])
        sent=training[i][j].split()
        for words in sent:
            if words in model_dict:
                temp1.append(list(model_dict[words]))
            else:
                temp1.append(list(np.zeros(100)))
    train.append(temp1)

In [13]:
#Padding for making the tensor
for i in range(len(train)):
    train[i].extend([[0]*100]*(2500-len(train[i])))

In [14]:
padded1=np.array(train)

In [15]:
padded1.shape

(2234, 2500, 100)

In [16]:
for i in range(len(y)):
    y[i].extend([[0]*10]*(2500-len(y[i])))

In [17]:
len(y[0][0])

10

In [18]:
X_train=padded1

In [19]:
X_train.shape

(2234, 2500, 100)

In [20]:
y_train=np.array(y)

In [21]:
y_train.shape

(2234, 2500, 10)

In [22]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Masking
from keras.layers import Bidirectional
 

C:\Users\Ashish\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [23]:
##########################Adding Validation Dataset####################################
os.chdir(r'H:\document_classification\Dataset\val-data')
filenames_val=glob.glob('*.txt')

In [24]:
path='H:\\document_classification\\Dataset\\val-data'
validation=[]
for i in range(len(filenames_val)):
    new_path=path+'\\'+filenames_val[i]
    with open(new_path,'r') as infile:
        data=infile.read()
        data=data.split('\n')
        train_val=[words for words in data if (words!='')&(words!='-----')]
        validation.append(train_val)

In [25]:
import numpy as np
import re
y_val=[]
for i in range(0,len(validation)):
    temp=[]
    for j in range(len(validation[i])):
        validation[i][j]=re.sub('[^A-Za-z0-9]',' ',validation[i][j])
        s=[j+1]*len(validation[i][j].split())
        temp.extend(s)
        k=np.array(temp)
        k=k.reshape(len(k),1)
        from sklearn.preprocessing import OneHotEncoder
        onehot_encoder = OneHotEncoder(sparse=False)
        k = onehot_encoder.fit_transform(k)
    #k=pad_sequences()
    y_val.append(k)
print(len(y_val))

151


In [26]:
temp2=[]
for i in range(0,len(y_val)):
    temp=[]
    for j in range(len(y_val[i])):
        temp1=list(y_val[i][j])
        temp1.extend([0]*(10-len(temp1)))
        temp.append(temp1)
    temp2.append(temp)

In [27]:
y_val=temp2

In [28]:
val=[]
for i in range(0,len(validation)):
    temp1=[]
    for j in range(len(validation[i])):
        validation[i][j]=validation[i][j].lower()
        #training[i][j]=re.sub('[^A-Za-z0-9]',' ',training[i][j])
        sent=validation[i][j].split()
        for words in sent:
            if words in model_dict:
                temp1.append(list(model_dict[words]))
            else:
                temp1.append(list(np.zeros(100)))
    val.append(temp1)

In [29]:
for i in range(len(val)):
    val[i].extend([[0]*100]*(2500-len(val[i])))

In [31]:
val_train=np.array(val)

In [32]:
for i in range(len(y_val)):
    y_val[i].extend([[0]*10]*(2500-len(y_val[i])))


In [33]:
val_test=np.array(y_val)

In [34]:
val_train.shape

(151, 2500, 100)

In [35]:
val_test.shape

(151, 2500, 10)

In [36]:
#Buiding the Bidirectional model for classification
model = Sequential()
model.add(Masking(mask_value=0., input_shape=(2500,100)))
model.add(Bidirectional(LSTM(100, input_shape=(2500,100), return_sequences=True)))
#model.add(Dense(50))
model.add(Dense(units=10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [37]:
X_train.shape

(2234, 2500, 100)

In [38]:
y_train.shape

(2234, 2500, 10)

In [39]:
val_train.shape

(151, 2500, 100)

In [40]:
val_test.shape

(151, 2500, 10)

In [41]:
model.fit(X_train, y_train,epochs=10, batch_size=1, verbose=1,validation_data=(val_train,val_test))

Train on 2234 samples, validate on 151 samples
Epoch 1/10
2234/2234 [==============================] - 7373s 3s/step - loss: 0.8203 - acc: 0.6436 - val_loss: 0.4373 - val_acc: 0.8125
Epoch 2/10
2234/2234 [==============================] - 7628s 3s/step - loss: 0.5059 - acc: 0.7869 - val_loss: 0.4049 - val_acc: 0.8243
Epoch 3/10
2234/2234 [==============================] - 7479s 3s/step - loss: 0.4619 - acc: 0.8055 - val_loss: 0.4812 - val_acc: 0.7967
Epoch 4/10
2234/2234 [==============================] - 7670s 3s/step - loss: 0.4312 - acc: 0.8210 - val_loss: 0.4104 - val_acc: 0.8203
Epoch 5/10
2234/2234 [==============================] - 7467s 3s/step - loss: 0.3946 - acc: 0.8361 - val_loss: 0.3914 - val_acc: 0.8291
Epoch 6/10
2234/2234 [==============================] - 7402s 3s/step - loss: 0.3704 - acc: 0.8457 - val_loss: 0.4102 - val_acc: 0.8247
Epoch 7/10
2234/2234 [==============================] - 7487s 3s/step - loss: 0.3446 - acc: 0.8586 - val_loss: 0.3983 - val_acc: 0.8364
E

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Ashish\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-41-f9e0dcb6f516>", line 1, in <module>
    model.fit(X_train, y_train,epochs=10, batch_size=1, verbose=1,validation_data=(val_train,val_test))
  File "C:\Users\Ashish\Anaconda3\lib\site-packages\keras\models.py", line 1002, in fit
    validation_steps=validation_steps)
  File "C:\Users\Ashish\Anaconda3\lib\site-packages\keras\engine\training.py", line 1705, in fit
    validation_steps=validation_steps)
  File "C:\Users\Ashish\Anaconda3\lib\site-packages\keras\engine\training.py", line 1236, in _fit_loop
    outs = f(ins_batch)
  File "C:\Users\Ashish\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py", line 2482, in __call__
    **self.session_kwargs)
  File "C:\Users\Ashish\Anaconda3\lib\site-packages\tensorflow\python\client\session.py", line 900, in ru

KeyboardInterrupt: 

In [63]:
classifier= model 

In [66]:
os.chdir("H:\\document_classification\\Dataset\\test-data")
path="H:\\document_classification\\Dataset\\test-data"
for dirpath,dirnames,filenames in os.walk(path):
    filname=filenames 

In [67]:
path="H:\\document_classification\\Dataset\\test-data"
test=[]
for i in range(len(filenames)):
    new_path=path+'\\'+filenames[i]
    with open(new_path,'r') as infile:
        data=infile.read()
        test.append(data)

In [68]:
file = r"H:\\glove.6B.100d.txt"
import numpy as np
def loadGloveModel(gloveFile):
    print ("Loading Glove Model")
    with open(gloveFile, encoding="utf8" ) as f:
       content = f.readlines()
    model = {}
    for line in content:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print ("Done.",len(model)," words loaded!")
    return model
model_dict=loadGloveModel(file)

Loading Glove Model
Done. 400000  words loaded!


In [70]:
original_test = []
new_test=[]
for i in range(0,len(test)):
    temp1=[]
    test[i]=test[i].lower()
    l=re.sub('[^A-Za-z0-9]',' ',test[i])
    temp2=l.split()
    original_test.append(temp2)
    for words in temp2:
        if words in model_dict:
            temp1.append(list(model_dict[words]))
        else:
            temp1.append(list(np.zeros(100)))
    new_test.append(temp1)

In [142]:
from keras.preprocessing.sequence import pad_sequences
padded_test=pad_sequences(new_test,padding='post',maxlen=2500)

In [71]:
for i in range(len(new_test)):
    new_test[i].extend([[0]*100]*(2500-len(new_test[i])))

In [72]:
X_test=np.array(new_test)

In [73]:
X_test.shape

(250, 2500, 100)

In [74]:
k=model.predict(X_test)

In [76]:
y_pred=[]
for j in range(len(k)):
    pred=[]
    for item in k[j]:
        temp=[]
        item=list(item)
        for items in item:
            if(items==max(item)):
                temp.append(1)
            else:
                temp.append(0)
        pred.append(temp)
    y_pred.append(pred)

In [77]:
corr_pred = []
for i in range(0,len(test)):
    corr_pred.append(y_pred[i][0:len(original_test[i])])

In [78]:
fin_pred=[]
for i in range(0,len(k)):
    output=list(map(sum,zip(*corr_pred[i])))
    l=0
    new_pred=[]
    for items in output:
        if(l<len(original_test[i])):
            l=l+items
            new_pred.append(l)
    fin_pred.append(new_pred[:-1])

In [79]:
dict_test=[]
for i in range(0,len(test)):
    test[i]=test[i].lower()
    l=re.sub('[^A-Za-z0-9]',' ',test[i])
    temp2=l.split()
    dict_test.append(temp2)

In [167]:
temp=[1]
for i in range(len(fin_pred[0])):
    r=dict_test[0][fin_pred[0][i]+1]
    s=dict_test[0][fin_pred[0][i]+2]
    t=dict_test[0][fin_pred[0][i]+3]
    u=dict_test[0][fin_pred[0][i]+4]
    pattern=r+r"[^A-Za-z0-9]*"+s+r"[^A-Za-z0-9]*"+t+r"[^A-Za-z0-9]*"+u
    #print(r,s,t)
    p=re.compile(pattern)
    for m in p.finditer(test[0]):
        k=m.start()
    temp.append(k-1)
#temp.extend(len(test[0])-1)
temp.append(len(test[0]))
temp

[1, 319, 993, 1476, 1558, 2008]

In [215]:
fin_pred[106]

[52, 113, 147, 2447]

In [80]:
y_final_result=[]
for j in range(250):
    #ank=ank+1
    #print(ank)
    temp=[1]
    for i in range(len(fin_pred[j])):
        #print(j)
        r=dict_test[j][fin_pred[j][i]+1]
        s=dict_test[j][fin_pred[j][i]+2]
        t=dict_test[j][fin_pred[j][i]+3]
        u=dict_test[j][fin_pred[j][i]+4]
        pattern=r+r"[^A-Za-z0-9]*"+s+r"[^A-Za-z0-9]*"+t+r"[^A-Za-z0-9]*"+u
        #print(r,s,t)
        p=re.compile(pattern)
        for m in p.finditer(test[j]):
            k=m.start()
        temp.append(k-1)
#temp.extend(len(test[0])-1)
    temp.append(len(test[j]))
    y_final_result.append(temp)

In [154]:
# In[590]:
y_final_result

[[1, 319, 993, 1476, 1558, 2008],
 [1, 314, 995, 1578, 1668, 2137],
 [1, 278, 1220, 2272, 2497, 2955],
 [1, 377, 857, 1613, 1717, 2154],
 [1, 404, 776, 1644, 1793, 2215],
 [1, 351, 828, 1425, 1542, 1993],
 [1, 307, 750, 1224, 1323, 1702],
 [1, 396, 701, 1383, 1523, 1991],
 [1, 377, 1120, 1950, 2118, 2726],
 [1, 326, 777, 1410, 1509, 1922],
 [1, 325, 1169, 2489, 2607, 3022],
 [1, 305, 809, 1395, 1489, 1925],
 [1, 263, 1032, 1410, 1525, 1873],
 [1, 286, 900, 1398, 1499, 1967],
 [1, 328, 873, 1602, 1691, 2140],
 [1, 336, 1084, 1938, 2127, 2655],
 [1, 265, 746, 1323, 1404, 1863],
 [1, 221, 875, 1298, 1370, 1862],
 [1, 309, 1085, 2098, 2252, 2704],
 [1, 324, 836, 1468, 1584, 2078],
 [1, 245, 815, 1357, 1479, 1918],
 [1, 203, 800, 1221, 1326, 1775],
 [1, 273, 774, 1367, 1503, 1965],
 [1, 265, 786, 1535, 1646, 2105],
 [1, 306, 917, 1779, 1913, 2326],
 [1, 272, 830, 1412, 1536, 1959],
 [1, 400, 1502, 2024, 2167, 2596],
 [1, 377, 941, 2248, 2419, 2797],
 [1, 359, 858, 1454, 1538, 2008],
 [1, 31

In [160]:
filenames

['test_1.txt',
 'test_10.txt',
 'test_100.txt',
 'test_101.txt',
 'test_102.txt',
 'test_103.txt',
 'test_104.txt',
 'test_105.txt',
 'test_106.txt',
 'test_107.txt',
 'test_108.txt',
 'test_109.txt',
 'test_11.txt',
 'test_110.txt',
 'test_111.txt',
 'test_112.txt',
 'test_113.txt',
 'test_114.txt',
 'test_115.txt',
 'test_116.txt',
 'test_117.txt',
 'test_118.txt',
 'test_119.txt',
 'test_12.txt',
 'test_120.txt',
 'test_121.txt',
 'test_122.txt',
 'test_123.txt',
 'test_124.txt',
 'test_125.txt',
 'test_126.txt',
 'test_127.txt',
 'test_128.txt',
 'test_129.txt',
 'test_13.txt',
 'test_130.txt',
 'test_131.txt',
 'test_132.txt',
 'test_133.txt',
 'test_134.txt',
 'test_135.txt',
 'test_136.txt',
 'test_137.txt',
 'test_138.txt',
 'test_139.txt',
 'test_14.txt',
 'test_140.txt',
 'test_141.txt',
 'test_142.txt',
 'test_143.txt',
 'test_144.txt',
 'test_145.txt',
 'test_146.txt',
 'test_147.txt',
 'test_148.txt',
 'test_149.txt',
 'test_15.txt',
 'test_150.txt',
 'test_151.txt',
 'tes

In [81]:
segments=[]
for i in range(len(y_final_result)):
    temp=[str(i) for i in y_final_result[i]]
    segments.append('-'.join(temp))

segments

['1-474-852-1234-1651-2008',
 '1-373-794-1308-1731-2137',
 '1-519-1083-1868-2207-2955',
 '1-512-991-1448-1770-2154',
 '1-476-826-1267-1729-2215',
 '1-483-818-1210-1612-1993',
 '1-493-876-1178-1469-1702',
 '1-425-770-1178-1571-1991',
 '1-473-1019-1792-1890-2726',
 '1-395-851-1264-1603-1922',
 '1-388-835-1641-2489-3022',
 '1-429-658-1186-1551-1925',
 '1-316-708-1001-1507-1873',
 '1-524-740-1279-1499-1967',
 '1-471-860-1332-1724-2140',
 '1-638-1041-1594-2099-2655',
 '1-518-977-1333-1631-1863',
 '1-466-784-1089-1456-1862',
 '1-394-940-1697-2208-2704',
 '1-320-714-1226-1528-2078',
 '1-434-874-1262-1521-1918',
 '1-412-806-1236-1548-1775',
 '1-350-682-1078-1493-1965',
 '1-378-786-1158-1622-2105',
 '1-335-817-1345-1758-2326',
 '1-440-706-1139-1536-1959',
 '1-579-1322-1789-2201-2596',
 '1-494-964-1464-2324-2797',
 '1-429-884-1248-1612-2008',
 '1-368-728-1149-1476-1943',
 '1-483-808-1276-2268-2639',
 '1-483-827-1140-1540-1919',
 '1-401-722-1046-1468-1789',
 '1-433-765-1186-1587-2122',
 '1-465-88

In [162]:
len(segments)

154

In [82]:
import pandas as pd

In [83]:
ankit_result= pd.DataFrame(
    {'file_name':filenames,
     'segments':segments
    })
ankit_result.to_csv('test_result_ankit_lenovo.csv')